#### PIPE - 5:    
This pipe covers the following : 
1. All features of pipe 4
2. Multi Thread implementation for image upscaling function.


In [ ]:
# threading test

In [ ]:
#### Create function to load models and trackers

In [10]:
%%time
import cv2
import numpy as np
import pandas as pd
import torch
import time
import threading 
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *
start_time=time.time()
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')

while True:    
    ret, im = vid.read()
    
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    im=line_plotter(frame=im,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(im, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(im, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy,bbox_list=plot_tracks(track,im)
               
                all_id_tracker.append(object_id)
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,im,all_frames_record_path)    
    frame_count+=1
    out_main.write(im)
    
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
end_time=time.time()
print(f'total_time {end_time-start_time}')

2024-05-15 11:55:37.739 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


total_time 10.350139141082764
CPU times: user 11.2 s, sys: 14 s, total: 25.1 s
Wall time: 10.4 s


In [2]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)
maintained=json_frame_order_checker(main_violation_json)
if maintained:
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)

order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
CPU times: user 209 ms, sys: 1.76 s, total: 1.97 s
Wall time: 2.07 s


In [3]:
%%time
if maintained:
    video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path)

CPU times: user 26 s, sys: 18.6 s, total: 44.5 s
Wall time: 12.6 s


In [4]:
%%time
if maintained:
    cropped_images_path_list=evidance_cropper_mt(evidance_img_dir_paths,main_violation_json)

CPU times: user 43.7 s, sys: 1.41 s, total: 45.1 s
Wall time: 3.38 s


In [5]:
cropped_images_path_list

['/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/5',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/10',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/28',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/34',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/72',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/88',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/107',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/113',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/174',
 '/home/hlink/workspace/learning/boxmot/14_05_2024_infenrence_output/cropped_evidance_images/169',
 '/home/hlink/wor

In [6]:
%%time
deblurred_image_paths=deblur_images(cropped_images_path_list,main_violation_json)

CPU times: user 570 ms, sys: 300 ms, total: 871 ms
Wall time: 908 ms


In [9]:
%%time
# using default 8 threads to achive max effiencicy
enhance_image_path=image_enhancement_using_limit_mpx(deblurred_image_paths,num_threads=8)

CPU times: user 4.35 s, sys: 4.68 s, total: 9.02 s
Wall time: 1h 38min 37s


In [ ]:
def video_clip_thread(entry,violation_frame_tracker,all_frames_record_path,window_size=20):
        previous_frames,post_frames=frame_picker(violation_frame_tracker[entry[0]],all_frames_record_path,window_size)
        input_frame_list=previous_frames+post_frames
        evidance_video_creator(input_frame_list,f'evidance_{entry[0]}',output_folder_path=entry[2])
def video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path):
    """
    Used to create and store video clips from the give frame paths to the given path
    """
    thread_list_=[]
    for i in zip(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths):    
        t=threading.Thread(target=video_clip_thread,args[i,violation_frame_tracker,all_frames_record_path])
        t.start()
        thread_list_.append(t)
    for j in thread_list_:
        j.join()
    return evidance_clip_dir_paths

In [ ]:
def timer_function(thread_number):
    start_time=time.time()
    print(f"Straing it : {start_time}")
    time.sleep(5)
    end_time=time.time()
    print(f"ending it at :{end_time}")
    total_time=end_time-start_time
    print(f"total_time for thread - {thread_number} is {total_time}s")

In [ ]:
for i in range(100):
    list_of_var=[]
    t=threading.Thread(target=timer_function,args=[i])
    t.start()
    list_of_var.append(t)
for j in list_of_var:
    j.join()

In [ ]:
from threading import Thread

def foo(bar):
        print('hello {0}'.format(bar))
        return "foo"

class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs, Verbose)
        self._return = None
    def run(self):
        if self._Thread__target is not None:
            self._return = self._Thread__target(*self._Thread__args,
                                                **self._Thread__kwargs)
    def join(self):
        Thread.join(self)
        return self._return

twrv = ThreadWithReturnValue(target=foo, args=('world!',))

twrv.start()
print(twrv.join())